# Hierarchical Topographic Factor Analysis
By Jeremy R. Manning ([jeremy.r.manning@dartmouth.edu](mailto:jeremy.r.manning@dartmouth.edu)) and Paxton C. Fitzpatrick ([Paxton.C.Fitzpatrick@dartmouth.edu](mailto:Paxton.C.Fitzpatrick@dartmouth.edu))

# Overview

In this demonstration, we'll be using the [BrainIAK](https://brainiak.org/) Python toolbox to apply [Hierarchical Topographic Factor Analysis (HTFA)](https://www.sciencedirect.com/science/article/abs/pii/S1053811918300715) to an fMRI dataset.

The demo will comprise three main steps:
1. Apply HTFA to the dataset to discover a basis set of network "nodes"
2. Apply a [dynamic correlation model](https://www.biorxiv.org/content/10.1101/763821v1.full.pdf) to the HTFA fits to characterize the network dynamics
3. Visualize the network dynamics in two ways:
  - An animated [chord diagram](https://en.wikipedia.org/wiki/Chord_diagram)
  - An animated brain plot
  
# Getting started

The easiest way to run this notebook is to download the corresponding Docker image.  That will install the necessary toolboxes and dependencies, and it will also download the data you'll be analyzing.

To build the docker image, navigate to this folder and run:
```
docker build --rm --force-rm -t htfa .
```

To start the image for the first time, run:
```
docker run -it -p 8888:8888 --name htfa -v $PWD:/mnt htfa
```
and on subsequent times, run:
```
docker start htfa && docker attach htfa
```

When the docker image is started, it will automatically start a Jupyter notebook server.  Copy and paste the *third* link into a browser to interact with this notebook.

To stop running the container, run:
```
docker stop htfa
```

# Initialization

Import libraries and helper functions and load the [dataset](https://drive.google.com/open?id=1IBA39ZZjeGS1u_DvZdiw1AZZQMS3K5q0).  The dataset we'll be analyzing is a subset of the story listening dataset collected by [Simony et al. (2016)](https://www.nature.com/articles/ncomms12141).

In [7]:
# the next two lines suppress a (meaningless) warning about the graphics backend
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

from brainiak.factoranalysis.htfa import HTFA
import holoviews as hv
import nibabel as nb
import nilearn as nl
import nltools as nlt
import timecorr as tc
import seaborn as sns

from mpi4py import MPI
from multiprocessing import Pool

import os, sys
from glob import glob as lsdir

from helpers import nii2cmu, cmu2nii, animate_chord, animate_connectome

%matplotlib inline

In [2]:
# data
intact = lsdir('/data/Pieman2/sub-*/func/*intact*.nii.gz')
scrambled = lsdir('/data/Pieman2/sub-*/func/*word*.nii.gz')

combined_fnames = list.copy(intact)
combined_fnames.extend(scrambled)
mask = '/data/Pieman2/masks/avg152T1_gray_3mm.nii.gz'

# Fit HTFA to data

First we need to convert the dataset into [CMU format](http://www.cs.cmu.edu/afs/cs/project/theo-73/www/science2008/README-data-documentation.txt).  Consistent with CMU format, nilearn expects the data matrices to have number-of-timepoints rows and number-of-voxels columns.  BrainIAK expects the data in the transpose of that format-- number-of-voxels by number-of-timepoints matrices. We can easily convert between the two formats using the map function.

After wrangling the data, we'll fit HTFA to the full dataset to identify network nodes.

In [3]:
# convert data to CMU format
cmu_data = [nii2cmu(f) for f in combined_fnames]

In [5]:
# convert to HTFA format
htfa_data = [{'R': x['R'], 'Z': x['Y'].T} for x in cmu_data]

In [8]:
# configure MPI:
#  - if the local environment supports MPI, do computations in parallel (fast!)
#  - if MPI is not supported, do computations in serial (slower)

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    import logging
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [9]:
# set up HTFA model
K = 25 #number of nodes; decrease to speed up analysis, increase for greater accuracy (min: 1)
nvoxels, ntimepoints = htfa_data[0]['Z'].shape

htfa = HTFA(K=K,
            n_subj=len(htfa_data),
            max_global_iter=5, #decrease for speed, increase for greater accuracy (min: 1)
            max_local_iter=2,  #decrease for speed, increase for greater accuracy (min: 1)
            voxel_ratio=0.5,   #decrease for speed, increase for greater accuracy (positive; max: 1.0)
            tr_ratio=0.5,      #decrease for speed, increase for greater accuracy (positive; max: 1.0)
            max_voxel=int(nvoxels*0.05), #decrease for speed, increase for greater accuracy (max: number of voxels)
            max_tr=int(ntimepoints*0.1), #decrease for speed, increase for greater accuracy (max: number of TRs)
            verbose=False)

In [ ]:
# fit HTFA to htfa_data
htfa.fit([x['Z'] for x in htfa_data], [x['R'] for x in htfa_data])

INFO:brainiak.factoranalysis.tfa:TFA converged at 0 iteration.
INFO:brainiak.factoranalysis.tfa:TFA converged at 1 iteration.
INFO:brainiak.factoranalysis.tfa:TFA converged at 0 iteration.
INFO:brainiak.factoranalysis.tfa:TFA converged at 0 iteration.
INFO:brainiak.factoranalysis.tfa:TFA converged at 1 iteration.
INFO:brainiak.factoranalysis.tfa:TFA converged at 1 iteration.
INFO:brainiak.factoranalysis.tfa:TFA converged at 1 iteration.
INFO:brainiak.factoranalysis.tfa:TFA converged at 1 iteration.
INFO:brainiak.factoranalysis.tfa:TFA converged at 0 iteration.
INFO:brainiak.factoranalysis.tfa:TFA converged at 0 iteration.
INFO:brainiak.factoranalysis.tfa:TFA converged at 0 iteration.
INFO:brainiak.factoranalysis.tfa:TFA converged at 1 iteration.
INFO:brainiak.factoranalysis.tfa:TFA converged at 0 iteration.
INFO:brainiak.factoranalysis.tfa:TFA converged at 0 iteration.
INFO:brainiak.factoranalysis.tfa:TFA converged at 0 iteration.
INFO:brainiak.factoranalysis.tfa:TFA converged at 0 ite

# Plotting HTFA global and local hub locations

We'll generate a plot where the global hub locations are shown in black, and each subject's "local" hub locations are shown in color (where each subject is assigned a different color). The hubs should be in similar (but not identical) locations across subjects.  Note: if the number of hubs or iterations is small, or if the voxel and/or timepoint subsampling is high, the final result will tend to be close to the initialized values (with little variation across subjects).  Increase `max_global_iter`, `max_local_iter`, `max_voxel`, and `max_tr` to achieve greater accuracy.

In [ ]:
#set the node display properties
colors = np.repeat(np.vstack([[0, 0, 0], sns.color_palette("Spectral", htfa.n_subj)]), K, axis=0)
colors = list(map(lambda x: x[0], np.array_split(colors, colors.shape[0], axis=0))) #make colors into a list #FIXME: simplify this...
sizes = np.repeat(np.hstack([np.array(50), np.array(htfa.n_subj*[20])]), K)

#extract the node locations from the fitted htfa model
global_centers = htfa.get_centers(htfa.global_posterior_)
local_centers = [htfa.get_centers(x) for x in np.array_split(htfa.local_posterior_, htfa.n_subj)]
centers = np.vstack([global_centers, np.vstack(local_centers)])

#make the plot
niplot.plot_connectome(np.eye(K*(1 + htfa.n_subj)), centers, node_color=colors, node_size=sizes);

# Compute dynamic correlations

The timeseries of activations for each node, for each participant provide a low-dimensional embedding of the original data that we can use to efficiently examine dynamic connectivity patterns.  Obtaining these embeddings requires some data wrangling.

In [ ]:
#The per-timepoint node weights (for all of the subjects) are stored in a single vector, htfa.local_weights_.
#(This is to enable message passing via MPI.)  We need to split up the weights by subject,
#and then reshape each subject's node weights into a K by n_timepoints matrix (which we
#take the transpose of, for convenience later on).
n_timepoints = [x['Z'].shape[1] for x in htfa_data] #number of timepoints for each person
inds = np.hstack([0, np.cumsum(np.multiply(K, n_timepoints))])
W = [htfa.local_weights_[inds[i]:inds[i+1]].reshape([K, n_timepoints[i]]).T for i in np.arange(htfa.n_subj)]

In [ ]:
# filter out intact vs. word subjects

# compute dynamic ISFC for intact, (word) scrambled

# Generate animated chord diagram

In [ ]:
# intact

# scrambled

# Generate animated brain network plot

In [ ]:
# intact

# scrambled